In [1]:
import import_ipynb
import tfm_generic_functions as traffic

# https://docs.python.org/3/library/os.html
import os

# http://scikit-image.org/docs/stable/api/api.html
import skimage

# https://docs.scipy.org/doc/numpy/reference/
import numpy as np

import keras

from keras import models

# Core Layers: https://keras.io/layers/core/
# Convolution Layers: https://keras.io/layers/convolutional/
from keras import layers

# https://matplotlib.org/api/index.html
import matplotlib

# https://matplotlib.org/api/_as_gen/matplotlib.pyplot.html#module-matplotlib.pyplot
import matplotlib.pyplot as plt

import statistics 
from statistics import mode 

importing Jupyter notebook from tfm_generic_functions.ipynb


Using TensorFlow backend.


In [2]:
IMG_SHAPE = (64, 64)
ROOT_PATH = os.getcwd()

In [3]:
def get_keras_model_detector(classes):
    # IMPLEMENTACIÓN RED NEURONAL
    # En Keras la envoltura para cualquier red neuronal se crea con la clase Sequential
    model = models.Sequential()

    model.add(layers.Conv2D(32, (5, 5),
                            activation='relu', input_shape=(IMG_SHAPE[0], IMG_SHAPE[1], 3)))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Conv2D(64, (5, 5), activation='relu'))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Conv2D(128, (5, 5), activation='relu'))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(classes, activation='softmax'))
    return model

def get_keras_model_classificator(classes):
    # IMPLEMENTACIÓN RED NEURONAL
    # En Keras la envoltura para cualquier red neuronal se crea con la clase Sequential
    model = models.Sequential()

    model.add(layers.Conv2D(32, (5, 5),
                            activation='relu', input_shape=(IMG_SHAPE[0], IMG_SHAPE[1], 3)))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Conv2D(64, (5, 5), activation='relu'))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Conv2D(128, (5, 5), activation='relu'))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(classes, activation='softmax'))
    return model

In [4]:
def load_trained_model_detector(weights_path):
    model = get_keras_model_detector(2)
    model.load_weights(weights_path)
    return model

def load_trained_model_classificator(weights_path, classes):
    model = get_keras_model_classificator(classes)
    model.load_weights(weights_path)
    return model

In [5]:
labels_path = os.path.join(ROOT_PATH, "dataset_bel/labels.csv")
titles = t <raffic.read_csv(labels_path, ",")

In [6]:
model_detector = load_trained_model_detector(os.path.join(ROOT_PATH, "model_data_aug_64_64_det.h5"))
model_detector.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=['accuracy'])

model_classificator = load_trained_model_classificator(os.path.join(ROOT_PATH, "model_data_aug_64_64_bel.h5"),
                                                      len(titles))

model_classificator.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
images_test_path = [os.path.join(ROOT_PATH, "images_test/bike_002.jpg"),
                    os.path.join(ROOT_PATH, "images_test/bike_007.jpg"),
                    os.path.join(ROOT_PATH, "images_test/pedestrian_008.jpg")]

In [8]:
def get_tiles(image, tile_shape, offset):
    tiles = []
    xx = tile_shape[0]
    yy = tile_shape[1]
    for x in range(0, image.shape[0]- xx, offset):
        for y in range(0, image.shape[1] - yy, offset):
            tile = image[x:x + xx, y:y + yy]
            tiles.append(tile)
    return tiles

In [9]:
def get_normalized_image(image_path):
    image_test = skimage.data.imread(image_path)
    height = image_test.shape[0]
    width = image_test.shape[1]
    return skimage.transform.resize(image_test, (height, width), mode='constant')

In [10]:
signs = []
for image_path in images_test_path:
    image_test = get_normalized_image(image_path)
    image_tiles = get_tiles(image_test, IMG_SHAPE, 1)
    print("Tiles: {}".format(len(image_tiles)))
    signs_aux = []
    for tile in image_tiles:
        np_tile = np.expand_dims(tile, axis=0)
        result = model_detector.predict_classes(np_tile)
        if result == 1:
            result_sign = model_classificator.predict_classes(np_tile)
            signs_aux.append(result_sign[0])
    signs.append(signs_aux)

Tiles: 60976
Tiles: 60976
Tiles: 60976


In [11]:
for d in signs:
    print("Moda: {} Señal {}".format(mode(d), titles[mode(d)]))

Moda: 39 Señal ['39', 'D9']
Moda: 35 Señal ['35', 'D1b']
Moda: 56 Señal ['56', 'F49']
